In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data_features = pd.read_csv("/kaggle/input/dengai-dataset/DengAI_Predicting_Disease_Spread_-_Training_Data_Features.csv")
train_data_features['cases'] = pd.read_csv("/kaggle/input/dengai-dataset/DengAI_Predicting_Disease_Spread_-_Training_Data_Labels.csv", usecols=[3])
test_data_features = pd.read_csv("/kaggle/input/dengai-dataset/DengAI_Predicting_Disease_Spread_-_Test_Data_Features.csv")

In [ ]:
train_data_features

In [ ]:
(train_data_features.isnull().sum()/len(train_data_features))*100

In [ ]:
(test_data_features.isnull().sum()/len(test_data_features))*100

In [ ]:
train_data_features.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)
test_data_features.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)

In [ ]:
def correlation_heatmap(train):
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(15,15))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();

In [ ]:
ndvi_variables = train_data_features[['ndvi_ne', 'ndvi_nw', 'ndvi_sw', 'ndvi_se']]
correlation_heatmap(ndvi_variables)

In [ ]:
precipitation_variables = train_data_features[['precipitation_amt_mm', 'reanalysis_sat_precip_amt_mm', 'station_precip_mm']]
correlation_heatmap(precipitation_variables)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
temperature = train_data_features[['reanalysis_air_temp_k', 'reanalysis_avg_temp_k','reanalysis_dew_point_temp_k',
                                  'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k','reanalysis_tdtr_k','station_avg_temp_c',
                                   'station_diur_temp_rng_c', 'station_max_temp_c','station_min_temp_c']]
temperature = min_max_scaler.fit_transform(temperature)
temperature = pd.DataFrame(temperature)
correlation_heatmap(temperature)

In [ ]:
corr_matrix = train_data_features.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.98)]
del upper

print('drop SIMILAR columns:', to_drop)
train_data_features.drop(to_drop,1,inplace=True)